In this notebook we will train a <code>Word2Vec</code> model, aproximated nearest neighbour model and test it on local validation set.

We will use the <code>gensim</code> library for <code>Word2Vec</code>, aproximated nearest neighbour model form <code>annoy</code>.

For this purpose, I used the [dataset](https://www.kaggle.com/datasets/radek1/otto-full-optimized-memory-footprint) provided by Radek Osmulski. The prepared sets allow you to test the developed recommendation system. The developed code in this notebook is based on the example prepared by Radek (see [nootebook](https://www.kaggle.com/code/radek1/word2vec-how-to-training-and-submission)). To improve the result obtained by the model, we introduce some modifications. For tips related to these parameters please see [notebook](https://www.kaggle.com/code/balaganiarz0/word2vec-model-local-validation).

# Data preprocessing

In [1]:
!pip install polars
import gc
import polars as pl
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

train = pl.read_parquet('../input/otto-full-optimized-memory-footprint/train.parquet')
test = pl.read_parquet('../input/otto-full-optimized-memory-footprint/test.parquet')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 34.4 MB/s eta 0:00:00


In [2]:
sentences_df =  pl.concat([train, test]).groupby('session').agg(
    pl.col('aid').alias('sentence')
)

sentences = sentences_df['sentence'].to_list()
del sentences_df; gc.collect() 

0

# Training word2vec model

In [3]:
%%time
##模型训练
w2vec = Word2Vec(sentences=sentences, vector_size= 64, window = 3, negative = 8, ns_exponent = 0.2, sg = 1, min_count=1, workers=4)

CPU times: user 6h 1min 46s, sys: 35.2 s, total: 6h 2min 21s
Wall time: 1h 35min 18s


In [4]:
##模型保存
import pickle
ff = open('w2vmodel/w2vec.pkl','wb')
pickle.dump(w2vec,ff)
ff.close()